In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
car_kick=pd.read_csv('car_kick3.csv')

In [3]:
car_kick.head()

,PurchDate,Year,Age,Mileage,ActnAvgPrice,ActnClnPrice,RetlAvgPrice,RetlClnPrice,WarrantyCost,Auction,...,Transmission,WheelTypeID,WheelType,Nationality,Size,TopThreeAmericanName,VNZIP1,VNST,IsOnlineSale,Class
0,2010-11-17,2006,4,51954,6197.0,7062.0,9605.0,10426.0,1086.0,Manheim,...,Auto,2,Covers,American,Medium,Chrysler,32124,FL,0,0
1,2009-05-19,2005,4,89127,3688.0,4783.0,4483.0,5666.0,983.0,Adesa,...,Auto,2,Covers,American,Medium,Ford,78754,TX,0,0
2,2009-07-22,2006,3,71271,6897.0,8449.0,7949.0,9625.0,1974.0,Manheim,...,Auto,1,Alloy,American,Large,Gm,80011,CO,0,0
3,2010-09-29,2008,2,83338,7878.0,8925.0,11723.0,13026.0,2152.0,Manheim,...,Auto,2,Covers,American,Large,Gm,94544,CA,0,0
4,2009-03-18,2007,2,58698,8800.0,10091.0,10004.0,11398.0,1500.0,Adesa,...,Auto,1,Alloy,American,Large,Gm,77086,TX,0,0


In [4]:
car_kick.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66560 entries, 0 to 66559
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   PurchDate             66560 non-null  object 
 1   Year                  66560 non-null  int64  
 2   Age                   66560 non-null  int64  
 3   Mileage               66560 non-null  int64  
 4   ActnAvgPrice          66560 non-null  float64
 5   ActnClnPrice          66560 non-null  float64
 6   RetlAvgPrice          66560 non-null  float64
 7   RetlClnPrice          66560 non-null  float64
 8   WarrantyCost          66560 non-null  float64
 9   Auction               66560 non-null  object 
 10  Make                  66560 non-null  object 
 11  Model                 66560 non-null  object 
 12  Trim                  66560 non-null  object 
 13  SubModel              66560 non-null  object 
 14  Color                 66560 non-null  object 
 15  Transmission       

First, we need to drop the 'Year' column, because it represents the same information as age column. We don't need the 'PurchDate column as well.

In [5]:
car_kick.drop(['Year', 'PurchDate'], axis=1, inplace=True)

Then, we need to choose a price column from the 4 price columns in our dataset. The best option would be 'ActnClnPrice', because it represents the price for which the car was sold at auction.But before dropping other price columns, we can create a new column which will represent the auction clean price and retail clean price ratio.

In [6]:
car_kick['price_ratio']=round((car_kick['ActnClnPrice']/car_kick['RetlClnPrice']), 2)
car_kick.drop(['ActnAvgPrice', 'RetlAvgPrice', 'RetlClnPrice'], axis=1, inplace=True)
car_kick.rename(columns={'ActnClnPrice': 'Price'}, inplace=True)

In [7]:
car_kick['Make'].value_counts()

Chevrolet     16375
Dodge         12303
Ford          10576
Chrysler       8027
Pontiac        3743
Kia            2224
Nissan         1948
Hyundai        1703
Saturn         1669
Jeep           1546
Toyota         1084
Mitsubishi      955
Mazda           875
Mercury         789
Buick           675
Gmc             608
Honda           460
Suzuki          272
Oldsmobile      221
Isuzu           128
Volkswagen      118
Scion           104
Volvo            36
Lincoln          30
Mini             24
Subaru           24
Acura            23
Cadillac         14
Infiniti          3
Plymouth          2
Lexus             1
Name: Make, dtype: int64

For the 'Make' column we can rename all the values that appear less than 50 times into a separate group called 'Other'

In [8]:
car_kick['Make'].replace(['Volvo', 'Lincoln', 'Mini', 'Subaru', 'Acura', 'Cadillac', 'Infiniti', 'Plymouth', 'Lexus'], 'Other', inplace=True)
car_kick['Make'].value_counts()

Chevrolet     16375
Dodge         12303
Ford          10576
Chrysler       8027
Pontiac        3743
Kia            2224
Nissan         1948
Hyundai        1703
Saturn         1669
Jeep           1546
Toyota         1084
Mitsubishi      955
Mazda           875
Mercury         789
Buick           675
Gmc             608
Honda           460
Suzuki          272
Oldsmobile      221
Other           157
Isuzu           128
Volkswagen      118
Scion           104
Name: Make, dtype: int64

Next, we need to drop the 'Model', 'Trim' and 'SubModel' columns, they have too many unique values, which are just names of the models and trims and they are different for every car manufacturer. Instead, we have information about the size, colour and transmission of the cars in other columns.

In [9]:
car_kick.drop(['Model', 'Trim', 'SubModel'], axis=1, inplace=True)

We also need to drop the 'WheelTypeID' column, because as we mentioned earlier it represents the same information as 'WheelType' column with numeric values. 

In [10]:
car_kick.drop('WheelTypeID', axis=1, inplace=True)

The 'VNZIP1' column represents the zip code of the car registration, but we also have the state information in 'VNST' column, so we can drop the zip codes.

In [11]:
car_kick.drop('VNZIP1', axis=1, inplace=True)
car_kick.rename(columns={'VNST':'State'}, inplace=True)

In [12]:
car_kick.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66560 entries, 0 to 66559
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   66560 non-null  int64  
 1   Mileage               66560 non-null  int64  
 2   Price                 66560 non-null  float64
 3   WarrantyCost          66560 non-null  float64
 4   Auction               66560 non-null  object 
 5   Make                  66560 non-null  object 
 6   Color                 66560 non-null  object 
 7   Transmission          66560 non-null  object 
 8   WheelType             66560 non-null  object 
 9   Nationality           66560 non-null  object 
 10  Size                  66560 non-null  object 
 11  TopThreeAmericanName  66560 non-null  object 
 12  State                 66560 non-null  object 
 13  IsOnlineSale          66560 non-null  int64  
 14  Class                 66560 non-null  int64  
 15  price_ratio        

Next, we need to create dummy variables for our categorical columns.

In [13]:
car_kick_d=pd.get_dummies(car_kick)

In [14]:
car_kick_d.iloc[:,0:4].head()

,Age,Mileage,Price,WarrantyCost
0,4,51954,7062.0,1086.0
1,4,89127,4783.0,983.0
2,3,71271,8449.0,1974.0
3,2,83338,8925.0,2152.0
4,2,58698,10091.0,1500.0


Then we need to scale standardize the magnitude of numeric features. But before that, we need to separate the columns to standardize.

In [17]:
cols=['Age', 'Mileage', 'Price', 'WarrantyCost', 'price_ratio']
car_kick_d.replace([np.inf, -np.inf], np.nan, inplace=True)

scaler=StandardScaler()
scaled_cc=scaler.fit_transform(car_kick_d[cols])
sc_car_kick=pd.DataFrame(scaled_cc, columns=cols)
sc_car_kick.head()

,Age,Mileage,Price,WarrantyCost,price_ratio
0,-0.09898,-1.374500,-0.160785,-0.333448,-0.641444
1,-0.09898,1.209197,-1.028333,-0.511550,0.768570
2,-0.68480,-0.031878,0.367206,1.202030,1.121073
3,-1.27062,0.806834,0.548405,1.509817,-0.553318
4,-1.27062,-0.905760,0.992267,0.382416,1.209199


Now we need to concat the dataframe above with the dataframe with dummy variables.

In [18]:
other_cols=car_kick_d.drop(cols, axis=1)

scaled_car_kick=sc_car_kick.merge(other_cols, right_index=True, left_index=True)
scaled_car_kick.head()

,Age,Mileage,Price,WarrantyCost,price_ratio,IsOnlineSale,Class,Auction_Adesa,Auction_Manheim,Auction_Other,...,State_OK,State_OR,State_PA,State_SC,State_TN,State_TX,State_UT,State_VA,State_WA,State_WV
0,-0.09898,-1.374500,-0.160785,-0.333448,-0.641444,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.09898,1.209197,-1.028333,-0.511550,0.768570,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,-0.68480,-0.031878,0.367206,1.202030,1.121073,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-1.27062,0.806834,0.548405,1.509817,-0.553318,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,-1.27062,-0.905760,0.992267,0.382416,1.209199,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


And finally, we need to split into testing and training datasets.

In [20]:
X=scaled_car_kick.drop('Class', axis=1)
y=scaled_car_kick['Class']

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.25)